In [1]:
import dhlab as dh
import pandas as pd
import dhlab.api.dhlab_api as api
import requests
BASE_URL = api.BASE_URL

wordcloud er ikke installert, kan ikke lage ordskyer


In [18]:
corpus = dh.Corpus(doctype="digibok", ddk="8*", limit = 500000, lang='nob', from_year = 1950, to_year=2022)

In [19]:
corpus.size

101367

In [4]:
def get_document_frequencies(urns = None, cutoff= 0, words = None):
    """Fetch frequency counts of ``words`` in documents (``urns``).
    Call the API :py:obj:`~dhlab.constants.BASE_URL` endpoint
    `/frequencies`.
    :param list urns: list of uniform resource name strings, for example:
        ``["URN:NBN:no-nb_digibok_2008051404065", "URN:NBN:no-nb_digibok_2010092120011"]``
    :param int cutoff: minimum frequency of a word to be counted
    :param list words: a list of words to be counted - if left None, whole document is returned.
    """
    params = locals()
    r = requests.post(f"{BASE_URL}/frequencies", json=params)
    result = r.json()
    # check if words are passed - return differs a bit
    if words is None:
        structure = dict()
        for u in result:
            try:
                structure[u[0][0]] = dict([(x[1], x[2]) for x in u])
            except IndexError:
                pass
        df = pd.DataFrame(structure)
        df = df.sort_values(by=df.columns[0], ascending=False).fillna(0)
    else:
        df = pd.DataFrame(result)
        df.columns = ["urn", "word", "count", "urncount"]
        #df = pd.pivot_table(df, values="count", index="word", columns="urn").fillna(0)
    return df

In [20]:
def countwords(urns=None, words=None):
    hifreq = get_document_frequencies(urns=list(urns), words= words)
    wordcounts = pd.pivot_table(hifreq, values="count", index="word", columns="urn").fillna(0).transpose()
    uc = pd.pivot_table(hifreq, values="urncount", index="word", columns="urn").fillna(0)
    urncounts = uc.max()
    c = pd.DataFrame()
    for x in wordcounts:
        c[x] = wordcounts[x]*100/urncounts
    return c

In [31]:
wc = countwords(urns = list(corpus.corpus.urn), words = "han hun henne ham hennes hans jeg meg du deg vi oss der dit".split())

In [29]:
wc3 = countwords(urns = list(corpus.corpus.urn), words = "han hun henne ham hennes hans".split())

In [30]:
wc3.sum(axis=1).describe()

count    99082.000000
mean         2.517593
std          1.306758
min          0.000459
25%          1.572796
50%          2.448588
75%          3.386231
max         10.432570
dtype: float64

In [32]:
wc.sum(axis=1).describe()

count    100471.000000
mean          5.033751
std           1.779675
min           0.000578
25%           3.923619
50%           5.114185
75%           6.262884
max          20.361991
dtype: float64

In [26]:
wc = countwords(urns = list(corpus.corpus.urn), words = "han hun jeg du vi der dit".split())

In [ ]:
df = 